In [188]:
import tensorflow
from tensorflow import keras 
import pandas as pd 
import numpy as  np 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [189]:
pip install -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [190]:
df=pd.read_csv(r'C:\Users\SDS\Downloads\diabetes.csv')

In [191]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [192]:
x=df.iloc[:,0:8]
y=df.iloc[:,-1]

In [193]:
scale=StandardScaler()

In [194]:
x_transform=scale.fit_transform(x)
x_transform

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [195]:
x_train,x_test,y_train,y_test=train_test_split(x_transform,y,test_size=0.2,random_state=42)

In [196]:
model=Sequential()

In [197]:
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

In [198]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
20/20 [==============================] - 1s 15ms/step - loss: 0.7017 - accuracy: 0.5635 - val_loss: 0.6513 - val_accuracy: 0.6558
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6425 - accuracy: 0.6678 - val_loss: 0.6054 - val_accuracy: 0.6753
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6018 - accuracy: 0.6906 - val_loss: 0.5722 - val_accuracy: 0.7273
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5687 - accuracy: 0.7036 - val_loss: 0.5519 - val_accuracy: 0.7338
Epoch 5/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5465 - accuracy: 0.7101 - val_loss: 0.5354 - val_accuracy: 0.7338
Epoch 6/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5277 - accuracy: 0.7215 - val_loss: 0.5264 - val_accuracy: 0.7208
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5145 - accuracy: 0.7313 - val_loss: 0.5191 - val_accuracy: 0.7143
Epoch 8/100


In [199]:
import keras_tuner as kt

In [200]:
def bulid_model(hp):
    model1=Sequential()
    model1.add(Dense(32,activation='relu',input_dim=8))
    model1.add(Dense(1,activation='sigmoid'))
    optimizer=hp.Choice('optimizer',values=['Adam','SGD','RMSprop','AdamW','Adadelta','Adagrad','Adamax','Adafactor','Nadam','Ftrl'])
    model1.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    return model1

In [201]:
tuner=kt.RandomSearch(bulid_model,objective='val_accuracy',max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [202]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [203]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'Adam'}

In [204]:
# calculate how much nodes in layers 

In [205]:
def bulid_models(hp):
    model=Sequential()
    units=hp.Int('units',min_value=8,max_value=128,step=8)
    model.add(Dense(units=units,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [206]:
tuner=kt.RandomSearch(bulid_models,
objective='val_accuracy',max_trials=5,
directory='mydir',
project_name='how_much_node',)

Reloading Tuner from mydir\how_much_node\tuner0.json


In [207]:
tuner.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [208]:
tuner.get_best_hyperparameters()[0].values

{'units': 104}

In [209]:
model=tuner.get_best_models(num_models=1)[0]
model.fit(x_train,y_train,epochs=200,validation_data=(x_test,y_test),initial_epoch=6)

Epoch 7/200
20/20 [==============================] - 1s 15ms/step - loss: 0.4796 - accuracy: 0.7752 - val_loss: 0.4971 - val_accuracy: 0.8052
Epoch 8/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4691 - accuracy: 0.7801 - val_loss: 0.4935 - val_accuracy: 0.7857
Epoch 9/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4615 - accuracy: 0.7752 - val_loss: 0.4938 - val_accuracy: 0.7727
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4559 - accuracy: 0.7785 - val_loss: 0.4964 - val_accuracy: 0.7727
Epoch 11/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4511 - accuracy: 0.7801 - val_loss: 0.4966 - val_accuracy: 0.7662
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4467 - accuracy: 0.7801 - val_loss: 0.4972 - val_accuracy: 0.7727
Epoch 13/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4429 - accuracy: 0.7850 - val_loss: 0.4946 - val_accuracy: 0.7727
Epoch 14

In [210]:
# how much hidden layers 

In [211]:
def models(hp):
    model=Sequential()
    model.add(Dense(104,activation='relu',input_dim=8))
    for i in range(hp.Int('hidden_lyers',1,10)):
        model.add(Dense(units='units'+str(i),min_value=8,max_value=512,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model
   

In [212]:
tunn=kt.RandomSearch(bulid_models,
directory='layers',
project_name='hidden_layers',
objective='val_accuracy',
max_trials=5)

Reloading Tuner from layers\hidden_layers\tuner0.json


In [213]:
tunn.search(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

In [214]:
tunn.get_best_hyperparameters()[0].values

{'units': 104}

In [215]:
model1=tunn.get_best_models(num_models=1)[0]

In [216]:
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 104)               936       
                                                                 
 dense_1 (Dense)             (None, 1)                 105       
                                                                 
Total params: 1041 (4.07 KB)
Trainable params: 1041 (4.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [217]:
tunn.results_summary()

Results summary
Results in layers\hidden_layers
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 104
Score: 0.798701286315918

Trial 2 summary
Hyperparameters:
units: 120
Score: 0.7792207598686218

Trial 3 summary
Hyperparameters:
units: 88
Score: 0.7727272510528564

Trial 1 summary
Hyperparameters:
units: 16
Score: 0.7337662577629089

Trial 4 summary
Hyperparameters:
units: 8
Score: 0.5


In [218]:
def build_model(hp):
    model =Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='linear'))  # For regression problem we use linear activation fuction
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model
    

In [219]:
uner = kt.RandomSearch(
    build_model,
    objective='val_mean_absolute_error', # incase of classification it will be val_accuracy
    max_trials=5,
    executions_per_trial =3,
    directory='project',
    project_name='Air Quality Index')

Reloading Tuner from project\Air Quality Index\tuner0.json


In [220]:
uner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [221]:
uner.search(x_train, y_train,
             epochs=10,
             validation_data=(x_test, y_test))

In [222]:
uner.results_summary()

Results summary
Results in project\Air Quality Index
Showing 10 best trials
Objective(name="val_mean_absolute_error", direction="min")

Trial 4 summary
Hyperparameters:
num_layers: 14
units_0: 320
units_1: 480
learning_rate: 0.0001
units_2: 288
units_3: 480
units_4: 224
units_5: 480
units_6: 96
units_7: 480
units_8: 416
units_9: 224
units_10: 320
units_11: 512
units_12: 512
units_13: 480
units_14: 32
units_15: 352
units_16: 416
units_17: 288
units_18: 192
units_19: 160
Score: 0.25509703159332275

Trial 1 summary
Hyperparameters:
num_layers: 20
units_0: 224
units_1: 64
learning_rate: 0.001
units_2: 96
units_3: 320
units_4: 480
units_5: 128
units_6: 160
units_7: 416
units_8: 224
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
units_14: 32
units_15: 32
units_16: 32
units_17: 32
units_18: 32
units_19: 32
Score: 0.2614907920360565

Trial 3 summary
Hyperparameters:
num_layers: 5
units_0: 224
units_1: 96
learning_rate: 0.0001
units_2: 480
units_3: 224
units_4: 384
units_5: 32


In [224]:
model2=uner.get_best_models(num_models=1)[0]

In [229]:
model2.fit(x_train,y_train,epochs=100,initial_epoch=11,validation_data=(x_test,y_test))

Epoch 12/100
20/20 [==============================] - 5s 54ms/step - loss: 0.2586 - mean_absolute_error: 0.2586 - val_loss: 0.2543 - val_mean_absolute_error: 0.2543
Epoch 13/100
20/20 [==============================] - 1s 39ms/step - loss: 0.2424 - mean_absolute_error: 0.2424 - val_loss: 0.2673 - val_mean_absolute_error: 0.2673
Epoch 14/100
20/20 [==============================] - 1s 40ms/step - loss: 0.2361 - mean_absolute_error: 0.2361 - val_loss: 0.2516 - val_mean_absolute_error: 0.2516
Epoch 15/100
20/20 [==============================] - 1s 40ms/step - loss: 0.2248 - mean_absolute_error: 0.2248 - val_loss: 0.2666 - val_mean_absolute_error: 0.2666
Epoch 16/100
20/20 [==============================] - 1s 37ms/step - loss: 0.2092 - mean_absolute_error: 0.2092 - val_loss: 0.2522 - val_mean_absolute_error: 0.2522
Epoch 17/100
20/20 [==============================] - 1s 39ms/step - loss: 0.1984 - mean_absolute_error: 0.1984 - val_loss: 0.2534 - val_mean_absolute_error: 0.2534
Epoch 18/1